Übung : Container Life-Cycle
-----------------------------------------


### Auswahl der Image Registry (Dev + Ops)

Wählt eine Image Registry aus, in unserem Beispiel verwenden wir wieder https://hub.docker.com/.

Falls, noch nicht erfolgt, Erstellt einen Accounts auf https://hub.docker.com/.

Und dann:
* Username und Password als Umgebungsvariablen setzen (als Python Script)
* `docker login` ausführen

In [ ]:
import os
name = input("Username: ")
os.environ["USER"] = name

from getpass import getpass
password = getpass("Password: ")
os.environ["PW"] = password

In [ ]:
! docker login -u ${USER} -p ${PW}

- - -
### Erstellung des Images (Dev)

Dazu erstellen wir zuerst unseren (Python) Microservice `order`

In [ ]:
%%bash
cat <<'EOF' > order.py
from flask import Flask, jsonify, render_template_string
import socket

app = Flask(__name__)

orders = [
    {"id": 1, "customer": "Alice", "product": "Book", "quantity": 2, "price": 10},
    {"id": 2, "customer": "Bob", "product": "Pen", "quantity": 5, "price": 2},
    {"id": 3, "customer": "Charlie", "product": "Notebook", "quantity": 1, "price": 5}
]

@app.route('/order')
def order_page():
    total = sum(o["quantity"] * o["price"] for o in orders)
    html = """
    <html><body>
    <h2>Orders on {{host}}</h2>
    <table border=1>
      <tr><th>ID</th><th>Customer</th><th>Product</th><th>Qty</th><th>Price</th><th>Total</th></tr>
      {% for o in orders %}
        <tr><td>{{o.id}}</td><td>{{o.customer}}</td><td>{{o.product}}</td>
        <td>{{o.quantity}}</td><td>${{o.price}}</td><td>${{o.quantity*o.price}}</td></tr>
      {% endfor %}
      <tr><td colspan=5><b>Sum</b></td><td><b>${{total}}</b></td></tr>
    </table>
    </body></html>
    """
    return render_template_string(html, orders=orders, total=total, host=socket.gethostname())

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)
EOF

Und für die Paketierung des Microservices, als Container Image, ein Dockerfile

In [ ]:
%%bash
cat <<%EOF% >Dockerfile
FROM python:3.9-slim

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY order.py /app

# Install any needed packages specified in requirements.txt
RUN pip install Flask setproctitle requests prometheus_client

USER 1001:100

# Run order_service.py when the container launches
CMD ["python", "order.py"]
%EOF%

Beides zusammen ergibt mit `docker build` unser Container Image.

In [ ]:
%%bash
docker build -t index.docker.io/${USER}/order-k8s .

Das Resultat ist ein neues Image und das, dass Base Image (php) heruntergeladen wurde:

In [ ]:
! docker image ls

- - -
### Speichern des Images in der Registry (Dev)

Wenn wir sicher, sind das alles läuft Pushen wir das Container Image in unsere Registry.

In [ ]:
! docker push index.docker.io/${USER}/order-k8s

***
# Produktion (Ops)

Das Operation, holt das Container Image von der Registry und startet dieses mittels Kubernetes.

In [ ]:
! kubectl run order-k8s --image=${USER}/order-k8s --restart=Never
! kubectl expose pod/order-k8s --type="LoadBalancer" --port 8080
! echo "http://"$(cat ~/work/server-ip)":$(kubectl get service order-k8s -o=jsonpath='{ .spec.ports[0].nodePort }')/order"

***
Aufräumen

In [ ]:
! kubectl delete pod/order-k8s
! kubectl delete service/order-k8s